### Masking a raster array with shapely

Although you can use `fiona` to mask a raster as you read it in with `rasterio`, I wanted to use the mask on arrays that have already been loaded. This works with a regular grid and a polygon that is completely enclosed in the raster.

TODO - Test with polygons that extend beyond the raster.

In [1]:
from shapely.geometry import Polygon, Point
import shapely.vectorized
import json
# import fiona
import numpy as np
import rasterio # rasterio and OGR don't mix...see rasterio docs
import rasterio.mask
import matplotlib.pyplot as plt
%matplotlib inline

### Use `rasterio` to read in a geotiff

Use `src.transform` to determine the coordinates of the corner points.  

In [2]:
# read in a .tif file...this will result in a 32-bit array
fdir = "E:\\2017_Karen_Sandwich_maps\\"
fnames = ("2016-01-22_SandwichTNB_PT_DEM_10cm_trimmed.tif")
# read band 1 (elevation...only band in these files) into specifed window
with rasterio.open(fdir+fnames) as src:
    raster = src.read(1,)

    # upper left corner
    x1,y1=src.transform*(0,0)
    # lower right corner
    x2,y2 = src.transform*(src.width, src.height)
    nx, ny = (src.width, src.height)

print(x1,y1)
print(x2,y2)

# Set missing values to NaN
raster[raster<=-99]=np.nan

RasterioIOError: E:\2017_Karen_Sandwich_maps\2016-01-22_SandwichTNB_PT_DEM_10cm_trimmed.tif: No such file or directory

In [3]:
print(np.shape(raster))
# next step could also be done with np.mgrid, but not quite as intuitively
X,Y = np.meshgrid(np.linspace(x1,x2,nx),\
                  np.linspace(y1,y2,ny))

print(np.shape(X))

(10226, 11987)
(10226, 11987)


### Read a polygon that was saved in geojson format and build a shapely polygon

This seems harder than it should be. This version assumes that there is only one `features` in the .geojson file. If you have multiple `features`, you have to do something like this:

```
for feature in data['features']:
    print(feature['geometry']['type'])
    #print(feature['geometry']['coordinates'])
    #print(feature['geometry'])
```

In [4]:
with open('owash_poly.geojson') as f:
    data = json.load(f)

# convert to array
a=np.squeeze(data['features'][0]['geometry']['coordinates'])
print(np.shape(a))
# shapely.geometry.Polygon requires a sequence of coordinate tuples
#(not an array)
poly = Polygon(tuple(map(tuple,a)))
# print(poly.wkt)
print("area of polygon: ",poly.area)

(11, 2)
area of polygon:  5986.593737889501


### Use shapely to make a boolean mask

This takes a while....

In [16]:
m = shapely.vectorized.contains(poly,X,Y)

In [ ]:
### Various options for applying the mask

In [49]:
print(np.shape(m))
print(np.min(m),np.max(m))
# this stays 32-bit
mras1 = m*raster
mm = (1*m)
mm[mm~=1]=np.nan
mras2 = (1*m)*raster
print(np.nansum(mras1),np.nansum(mras2))
print(np.nanmax(mras1),np.nanmax(mras2))
print(mras1.dtype,mras2.dtype,raster.dtype)

SyntaxError: invalid syntax (<ipython-input-49-afbb248f0ec7>, line 6)

In [45]:
np.nan*False

nan